In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [10]:
class Net(nn.Module):

    def __init__(self, *args, **kwargs):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(400, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)


    def forward(self, input):
        a1 = F.relu(self.conv1(input))
        a1 = F.max_pool2d(a1, (2,2))

        a2 = F.relu(self.conv2(a1))
        a2 = F.max_pool2d(a2, (2,2))
        a2 = torch.flatten(a2)

        a3 = F.relu(self.fc1(a2))

        a4 = F.relu(self.fc2(a3))

        output = self.fc3(a4)

        return output
        

In [12]:
#Test Forward
net = Net()
x = torch.rand(1, 3, 32, 32)
output = net(x)
output

tensor([-0.1007,  0.0032,  0.1017,  0.0414, -0.0244, -0.0392,  0.0906, -0.0655,
        -0.0542,  0.0082], grad_fn=<ViewBackward0>)

In [15]:
#Defining Loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.01)

In [17]:
#Load Data
import torchvision
from torchvision import transforms
from torch.utils.data import random_split, DataLoader

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

batch_size = 4
validation_split = 0.1  # Percentage of the training dataset to be used as validation

full_trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                             download=True, transform=transform)

train_size = int((1 - validation_split) * len(full_trainset))
val_size = len(full_trainset) - train_size
trainset, valset = random_split(full_trainset, [train_size, val_size])

trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)
valloader = DataLoader(valset, batch_size=batch_size, shuffle=False, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

print(f"Train dataset size: {len(trainset)}")
print(f"Validation dataset size: {len(valset)}")
print(f"Test dataset size: {len(testset)}")

100%|██████████| 170M/170M [00:08<00:00, 19.9MB/s] 


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Train dataset size: 45000
Validation dataset size: 5000
Test dataset size: 10000
